<a href="https://colab.research.google.com/github/tqth/tqth.github.io/blob/main/tgmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup DeepLogo2

In [ ]:
!pip install -q yacs


In [ ]:
!git clone https://github.com/satojkovic/DeepLogo2.git


In [ ]:
%cd DeepLogo2
!wget http://image.ntua.gr/iva/datasets/flickr_logos/flickr_logos_27_dataset.tar.gz
#cp /content/drive/MyDrive/DeepLogo2/train_data/flickr_logos_27_dataset.tar.gz .
!tar zxvf flickr_logos_27_dataset.tar.gz
%cd flickr_logos_27_dataset
!tar zxvf flickr_logos_27_dataset_images.tar.gz
%cd ..
!rm flickr_logos_27_dataset.tar.gz
!rm flickr_logos_27_dataset_images.tar.gz


In [ ]:
!pwd

In [ ]:
%cd ..

In [ ]:
%cd DeepLogo2
!python preproc_annot.py


In [ ]:
!pwd

In [ ]:
!python flickr2coco.py --mode train --output_dir flickr_logos_27_dataset
!python flickr2coco.py --mode test --output_dir flickr_logos_27_dataset


## Visualize data

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os


In [ ]:
coco = COCO('/content/DeepLogo2/flickr_logos_27_dataset/flickr_logos_27_train.json')


In [ ]:
# load and display image
catIds = coco.getCatIds(catNms=['Intel'])
imgIds = coco.getImgIds(catIds=catIds)


In [ ]:
print(catIds)
print(imgIds)


In [ ]:
!pwd

In [ ]:
img_id = imgIds[np.random.randint(0,len(imgIds))]
print('Image id {}'.format(img_id))

img = coco.loadImgs(img_id)[0]

img_name = os.path.join('flickr_logos_27_dataset', 'flickr_logos_27_dataset_images', img['file_name'])
print('Image name: {}'.format(img_name))

I = io.imread(img_name)
plt.figure()
plt.imshow(I)


In [ ]:
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds)
anns = coco.loadAnns(annIds)

plt.imshow(I)
coco.showAnns(anns, draw_bbox=True)


## Setup detr

In [ ]:
import torch


In [ ]:
!python delete_head_and_save.py


## Train!!

In [ ]:
!python detr/main.py \
  --dataset_file "flickr_logos_27" \
  --coco_path "flickr_logos_27_dataset" \
  --output_dir "outputs" \
  --resume "detr-r50_no-class-head.pth" \
  --epochs 3


## Display results

In [ ]:
!pip install detr
from detr.util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path('outputs/')]


In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)


In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)


In [ ]:
!ls outputs


In [ ]:
model = torch.hub.load('facebookresearch/detr',
                       'detr_resnet50',
                       pretrained=False,
                       num_classes=27)

#checkpoint = torch.load('outputs/checkpoint.pth',
#                        map_location='cpu')
checkpoint = torch.load('/content/drive/MyDrive/DeepLogo2/model_detr/checkpoint.pth', map_location='cpu')

model.load_state_dict(checkpoint['model'],
                      strict=False)

model.eval();


In [ ]:
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]


In [ ]:
import config


In [ ]:
cfg = config.get_cfg_defaults()


In [ ]:
finetuned_classes = cfg.CLASS_NAMES


In [ ]:
import torchvision.transforms as T

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b


In [ ]:
def plot_finetuned_results(pil_img, prob=None, boxes=None, save_dir=None, image_name=None):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if prob is not None and boxes is not None:
      for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=c, linewidth=3))
          cl = p.argmax()
          text = f'{finetuned_classes[cl]}: {p[cl]:0.2f}'
          ax.text(xmin, ymin, text, fontsize=15,
                  bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    if save_dir is not None and image_name is not None:
      plt.savefig(os.path.join(save_dir, image_name))
    plt.show()


In [ ]:
def filter_bboxes_from_outputs(outputs,
                               threshold=0.7):

  # keep only predictions with confidence above threshold
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold

  probas_to_keep = probas[keep]

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

  return probas_to_keep, bboxes_scaled


In [ ]:
def run_workflow(my_image, my_model, save_dir, image_name):
  # mean-std normalize the input image (batch-size: 1)
  img = transform(my_image).unsqueeze(0)

  # propagate through the model
  outputs = my_model(img)

  for threshold in [0.8]: #[0.9, 0.7]:

    probas_to_keep, bboxes_scaled = filter_bboxes_from_outputs(outputs,
                                                              threshold=threshold)

    plot_finetuned_results(my_image,
                           probas_to_keep,
                           bboxes_scaled, save_dir, image_name)


In [ ]:
from PIL import Image

#img_name = '/content/DeepLogo2/flickr_logos_27_dataset/flickr_logos_27_dataset_images/4763207899.jpg'
img_name = '/content/DeepLogo2/flickr_logos_27_dataset/flickr_logos_27_dataset_images/1690276845.jpg'

im = Image.open(img_name)

run_workflow(im,
            model, None, None)


## Inference with all test images

In [ ]:
import json


In [ ]:
with open('flickr_logos_27_dataset/flickr_logos_27_test.json', 'r') as f:
  test_data = json.load(f)


In [ ]:
test_image_names = [d['file_name'] for d in test_data['images']]


In [ ]:
save_dir = '/content/drive/MyDrive/DeepLogo2/result_images_epoch100_model'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)


In [ ]:
for test_image_name in test_image_names:
  test_image_full_name = os.path.join(cfg.IMAGE_DIR, test_image_name)
  im = Image.open(test_image_full_name)
  run_workflow(im, model, save_dir, test_image_name)
